**Decoding Heart Disease**

**Introduction**


Heart disease can refer to a wide range of diseases, but in this case refers to coronary artery disease which involves reduction of blood flow to the heart due to atherosclerosis. Atherosclerotic plaque can build up in the arteries, and if near the heart can lead to heart attacks and other complications. The causes aren’t exactly clear, but plaque gradually builds up and is accompanied by inflammation of the arterial walls, reducing blood flow to vital organs and extremities (John Hopkins Medicine, 2021).


The data set contains data from 4 databases, including Cleveland, Hungary, Switzerland, and Long Beach (Virginia), the data from Cleveland was the only one processed. It has 14 usable attributes, half of which are integer variables and the other half are categorical. We have attempted to answer the question, **what factors contributes to heart disease?**


Heart disease is of growing concern, especially as humans continue to grow older and has become the leading factor of death for humans around the world accounting for 16% of total deaths globally (WHO’s Global Health Estimates, 2020). It also disproportionately affects wealthier nations and more importantly in this investigation has correlations with many health metrics and biological markers (WHO’s Global Health Estimates, 2020). 

We have built a model that will take certain health markers and we will be classifying patients based on these health markers to predict whether or not they have heart disease.

In [1]:
#imports basic tools and the dataset below

import pandas as pd
import altair as alt
import numpy as np
!pip3 install -U ucimlrepo

In [2]:
from ucimlrepo import fetch_ucirepo

#imports dataset
heart_disease = fetch_ucirepo(name='Heart Disease')

In [3]:
#Sets up classifiers and diagnosis
classifiers = heart_disease.data.features 
diagnosis = heart_disease.data.targets 

#Creates dataframe and classifies diagnosis
heart_df = classifiers.assign(diagnosis = diagnosis)
heart_df['diagnosis'] = heart_df['diagnosis'].replace([1, 2, 3, 4], 1)

#Drops all columns except for predictors and diagnosis
heart_df_filter = heart_df[["age", "cp", "trestbps", "chol", "thalach", "diagnosis"]]

#Imports sk-learn tools
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, cross_validate
from sklearn.metrics import recall_score, confusion_matrix
from pandas import crosstab

#Sets a initial random seed to make data reproducable
np.random.seed(500)

heart_train, heart_test = train_test_split(heart_df_filter, train_size = 0.75)

#Grabs all the different diagnoses to get an initial table
heart_train_mean = heart_train.groupby("diagnosis").mean(numeric_only=True).reset_index()

display("Figure 1: Heart Disease grouped by diagnoses", heart_train_mean)

'Figure 1: Heart Disease grouped by diagnoses'

,diagnosis,age,cp,trestbps,chol,thalach
0,0,52.768595,2.809917,129.553719,244.438017,158.322314
1,1,56.924528,3.566038,133.500000,249.056604,140.566038


**Methods and Results**

Initially we can see a couple of trends just by grouping the data by diagnosis, most notably a null diagnosis being correlated with youth, lower chest pain (1 being asymptomatic), lower resting blood pressure and a higher maximum heart rate.

To further investigate a couple of links let's look at some graphs.

In [4]:
#Plot of age and heart disease

age = alt.Chart(heart_train).mark_bar().encode(
    x=alt.X("age", bin=alt.Bin(step=5)).title("Age"),
    y=alt.Y("count()").title("# of Patients"),
    color=alt.Color("diagnosis:N", title="Diagnosis")
).properties(title="Figure 2: Age and Heart Disease")


#Plot of max heart rate and heart disease

max_hr = alt.Chart(heart_train).mark_bar().encode(
    x=alt.X("thalach", bin=alt.Bin(step=20)).title("Max HR (Exercise)"),
    y=alt.Y("count()").title("# of Patients"),
    color=alt.Color("diagnosis:N", title="Diagnosis")
).properties(title="Figure 3: Max Heart Rate and Heart Disease")

display(age | max_hr)

alt.HConcatChart(...)

These visualizations show a couple of expected trends, the proportion of younger people 25-55 with a diagnosis is much lower than those 55 and up.

As for max heart rate there is significant negative correlation with max heart rate and heart disease. The 160-180 group is dominated by a null diagnosis, 140-160 has a roughly 50/50 split, but as we go lower the prevalence of heart disease becomes the norm, not the exception.

We will now setup a KNeighborsClassifier model below that will attempt to make predictions based off of the variables **age**, **trestbps**, **chol**, and **thalach** and spit out a target variable for **diagnosis**.

Since we are using KNeighborsClassifier the variables will have to be standardized for further analysis.

In order to find the best K value we will use GridSearchCV and do it in the range from 1-50 (inclusive). We will use a cv of 10 to maintain a solid balance between efficiency and performance.


In [5]:
#Creates the preprocessor
heart_pre = make_column_transformer(
    (StandardScaler(), ["age","cp","trestbps","chol","thalach"]),
    remainder="passthrough",
    verbose_feature_names_out=False
)

#Creates the model
knn = KNeighborsClassifier()

#Creates pipeline
heart_pipe = make_pipeline(heart_pre, knn)

#Fits training data onto the pipeline with diagnosis as the target variable
heart_pipe.fit(
    heart_train[["age","cp","trestbps","chol","thalach"]],
    heart_train["diagnosis"])

#Creates a grid and performs a grid search to attempt to find the best K value
grid = {
    "kneighborsclassifier__n_neighbors": range(1,51)
}

heart_grid = GridSearchCV(
    estimator = heart_pipe,
    param_grid = grid,
    cv = 10
)

#Fits it onto the training data
heart_grid.fit(
    heart_train[["age","cp","trestbps","chol","thalach"]],
    heart_train["diagnosis"])

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('standardscaler',
                                                                         StandardScaler(),
                                                                         ['age',
                                                                          'cp',
                                                                          'trestbps',
                                                                          'chol',
                                                                          'thalach'])],
                                                          verbose_feature_names_out=False)),
                                       ('kneighborsclassifier',
                                        KNeighborsClassifier())]),
             param_grid={'kneighborsclassifier__n_neighbors': range(1, 51)})

In [6]:
#Creates a dataframe out of the grid
heart_grid_df = pd.DataFrame(heart_grid.cv_results_)

#Creates a line plot out of the dataframe, using neighbors on the X and mean test score on the Y
grid_plot = alt.Chart(heart_grid_df, title="Figure 4: How Does Number of Neighbors Impact Test Score?").mark_line(point=True).encode(
    x=alt.X("param_kneighborsclassifier__n_neighbors").title("Number of Neighbors"),
    y=alt.Y("mean_test_score").title("Mean Test Score")
)

display(grid_plot)

alt.Chart(...)

Visualizing the number of neighbors above shows an interesting trend.

Looking at the line plot we can  see that increasing the number of neighbors significantly boosts the performance of the model, but it levels off very quickly. 

Lets sort the table by **rank_test_score** to find the best K values.

In [7]:
#Sorts the grid dataframe by rank_test_score
display("Figure 5: Best K Values",
        heart_grid_df.sort_values(by="rank_test_score").head(5)
        [["rank_test_score", "mean_test_score", "param_kneighborsclassifier__n_neighbors"]]
       )

'Figure 5: Best K Values'

,rank_test_score,mean_test_score,param_kneighborsclassifier__n_neighbors
37,1,0.718182,38
34,2,0.717589,35
47,3,0.713636,48
46,3,0.713636,47
31,3,0.713636,32


The best K value according to cross validation with 10 folds is $K=38$.

Now that we have a K value lets fit the training data with our new K value

We can then use our new fitted model to build predictions for our testing data.

In [8]:
#Sets K to 38
knn = KNeighborsClassifier(n_neighbors=38)

#Now fits the data using K = 38

#Creates pipeline
heart_pipe = make_pipeline(heart_pre, knn)

#Fits training data onto the pipeline with diagnosis as the target variable
heart_pipe.fit(
    heart_train[["age","cp","trestbps","chol","thalach"]],
    heart_train["diagnosis"])

#Predicts a diganosis for the testing data
heart_test = heart_test.assign(pred_diagnosis = 
                               heart_pipe.predict(heart_test[["age","cp","trestbps","chol","thalach"]]))

#Builds confusion matrix
conf_df = heart_test[["diagnosis","pred_diagnosis"]]
conf_df = conf_df.rename(columns={
    "diagnosis" : "Actual Diagnosis",
    "pred_diagnosis" : "Predicted Diagnosis"
})

conf_matrix = crosstab(conf_df["Actual Diagnosis"], conf_df["Predicted Diagnosis"])

display("Figure 6: How did we perform?", conf_matrix)

'Figure 6: How did we perform?'

Predicted Diagnosis,0,1
Actual Diagnosis,,
0,36,7
1,8,25


We can now evaluate our performance based on accuracy, precision and recall. 

With $61/76$ correct predictions total we achieved an $80.3\%$ accuracy.

With $25/32$ correct positive predictions we achieved an $78.1\%$ precision.

With $25/33$ true positives found we achieved a $75.8\%$ recall.

**Discussion**

After constructing a model with K-nearest neighbors to classify heart disease based on age, resting blood pressure, serum cholesterol and maximum heart rate we were able to achieve a recall of $75.8\%$ which suggests we could find roughly 3 in 4 patients with heart disease.

Given we are in the realm of healthcare we are most likely concerned with recall, especially when it comes to more severe pathologies like heart disease. The cost of accidentally misdiagnosing a false positive is likely not as severe as letting someone go untreated and have a false negative.

With an accuracy of $80.3\%$ this also suggests we usually made the right diagnosis 4 in 5 times. Although inadequate as a standalone diagnostic tool, this could be a good starting point as a first-line way of finding people who should seek further medical treatment. Operating on such limited data could also reduce the quality of our model, if expanded to include more patients we could possibly expect to build more accurate models.

These results suggest that the variables we chose (age, chest pain, resting blood pressure, serum cholesterol, and maximum heart rate) were good predictors of heart disease. Another possibility for improvement would be finding more health markers to include in our model, and would likely boost the performance of our model.

These findings allow for earlier intervention of heart disease through changes in lifestyle, medication management, and, if necessary, surgical procedures. Early detection and treatment can prevent or delay the progression of heart disease, reducing the risk of complications such as heart attacks, strokes, and heart failure.

This also poses questions about the future of medical diagnostics, especially given the emergence of machine learning in recent years. If we were able to build a relatively solid model off of limited data and tools what can advanced healthcare professionals use to help benefit them in screening and evaluating patients? Obviously we cannot replace trained doctors but as early detection systems predictive diagnostic models could be immensely useful in an already strained healthcare environment.

**References**

Janosi, A., Steinbrunn, W., Pfisterer, M., & Detrano, R. (1988, July 1). Heart Disease. UCI Machine Learning Repository. https://doi.org/10.24432/C52P4X.

John Hopkins Medicine. (2021). Atherosclerosis. Hopkins Medicine. https://www.hopkinsmedicine.org/health/conditions-and-diseases/atherosclerosis

WHO’s Global Health Estimates. (2020, December 9). The top 10 causes of death. World Health Organization; WHO. https://www.who.int/news-room/fact-sheets/detail/the-top-10-causes-of-death